<center> <h1> Gondret configuration : Probleme_FT_Disc_gen </h1></center>


In [ ]:
from trustutils import run
import matplotlib.pyplot as plt
run.introduction("Edouard Butaye")
run.TRUST_parameters()

## Problem description

This form allows comparing the trajectory of a particle that collide onto a wall in a dry (non-viscous) and wet (viscous) medium.  The particle is dropped and falls under the effect of gravity. The domain is a rectangular parallelepiped with a side length of 3 particle diameters and a height of 4 particle diameters.  
For the case of a dry collision, ie without any friction, the trajectory is compared to a analytical result.  
For the case of a wet collision, the trajectory is confronted to the experimental study of Gondret (2002) (https://hal.science/hal-02482943).

                                       imposed pressure and velocity
                                     ________________________________
                                    |                                |
                                    |                                |
                                    |          /-----------\         | 
                    imposed velociy |         /             \        | imposed velocity
                                    |        |    sperical   |       |
                                    |        |    particle   |    |  |
                                    |         \   dimeter D /   g |  |
                                    |          \-----------/      |  |
                                    |                            \ / |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |                                |
                                    |________________________________|

                                           imposed velocity


## Computation set up
### Geometric parameters
- Origin : -4.5e-3  0. -4.5e-3
- Node numbers : 16 21 16
- Lengths : 9e-3 12e-3 9e-3
### Physical parameters
- Particle :
      - mu  10 Pa.s
      - rho 7800 kg.m^{-3}
      - e_dry 0.97
      - radius  1.5e-3 m
- Fluid :
      - mu  1e-2 Pa.s
      - rho 935 
- Gravity : -9.81 m.s^{-2}
- Surface tension : 0 N/m (not considered for a solid particle) 
### Numerical parameters
- Time scheme : Euler explicite + diffusion implicite
- Threshold implicite diffusion : 10^{-11} 
- Time step : 3.75.10^{-5} s
- Total computation time : 10^{-3} s
- No remeshing (non-deformable particle)
- Velocity interpolation : VDF_lineaire

In [ ]:
run.reset()
run.initBuildDirectory()
dic_dry_collision = {
    "rho_f": "1.2e-3",
    "rho_p": "2150",
    "mu_f": "1.85e-5",
    "mu_p": "1.85e-2",
    "radius": "3e-3",
    "e_dry": "0.8",
    "Ox": "-9e-3",
    "Oy": "0.",
    "left": "-9e-3",
    "right": "9e-3",
    "h_dom": "24e-3",
    "length": "18e-3 24e-3 18e-3",
    "hinit": "9.78e-3",
    "collision_duration": "32e-6",
    "delta_t": "1e-6",
    "dt_sauv_lata": "1e-3",
    "tmax": "0.1",
    "e_dry": "0.8"
}

dic_wet_collision = {
    "rho_f": "935",
    "rho_p": "7800",
    "mu_f": "1e-2",
    "mu_p": "10",
    "radius": "1.5e-3",
    "Ox": "-4.5e-3",
    "Oy": "0.",
    "left": "-4.5e-3",
    "right": "4.5e-3",
    "h_dom": "12e-3",
    "length": "9e-3 12e-3 9e-3",
    "hinit": "8.88e-3",
    "collision_duration": "3.2e-4",
    "delta_t": "1.e-5",
    "dt_sauv_lata": "6.e-4",
    "tmax": "0.1",
    "e_dry": "0.97"
}

dic_hybrid_esi_model = {"collision_model": "hybrid_esi"}
dic_breugem_model = {"collision_model": "breugem"}

collision_type = {
    "dry": dic_dry_collision,
    "wet": dic_wet_collision
}

collision_model = {
    "hybrid_esi": dic_hybrid_esi_model,
    "breugem": dic_breugem_model
}

dic_color = {
    "hybrid_esi": "red",
    "breugem": "blue"
}

for collision, vcollision in collision_type.items():
    for model, vmodel in collision_model.items():
        target_repo = f"{collision}/{model}"
        run.addCaseFromTemplate("gondret.data", target_repo, {**vcollision,**vmodel},nbProcs=12)


run.printCases()

### Mesh generation

In [ ]:
for collision, vcollision in collision_type.items():
    for model, vmodel in collision_model.items():
            target_repo = f"{collision}/{model}"
            mesh_gen=run.TRUSTCase(target_repo,f"{run.ORIGIN_DIRECTORY}/src/mesh_template.data").copy("mesh.data")
            mesh_gen.substitute_template({**vcollision})
            run.executeCommand(f"cd {target_repo}; trust mesh.data")

Please note that the computation for all cases requires approximately an hour.

In [ ]:
run.runCases()

### Performances

In [ ]:
table = run.tablePerf()
table = table.drop(columns=["host", "system"]).drop("Total")
table

## Dry collision

### Computation of the particle vertical position and velocity
In the case of sedimentation in a non-viscous medium, the particle is subjected solely to gravity.
\begin{equation}
a_z=-g
\end{equation}
\begin{equation}
u_z=-gt+u_{z,0}
\end{equation}
\begin{equation}
y_z=-\frac{1}{2}gt^2+u_{z,0}t+h_0
\end{equation}
After the first rebound, the trajectory can be analyzed as a series of parabolic paths. For each parabola, $h_0=0$ and the initial velocity is calculated by multiplying the impact velocity by the restitution coefficient.

In [ ]:
import numpy as np
import pandas as pd
### params ###
radius=3e-3
drop_height=9.78e-3-radius
g=9.81
spatial_residual=1e-4
e_dry=0.8

### theoretical trajectory ###
def compute_particle_velocity(time,
                              initial_velocity):
    return (-g*time+initial_velocity)

def compute_particle_height(time,
                            initial_velocity,
                            initial_height):
    return (-1/2*g*time**2+initial_velocity*time+initial_height)
    
final_time=0.1
dt=1e-6
time_th=[]
position_th=[]
velocity_th=[]

initial_velocity=0.
initial_height=drop_height
relative_time=0
time=0.
glob_time=0.
ind_time=0
while (glob_time<final_time):
    vel=compute_particle_velocity(time,
                                  initial_velocity)
    pos=compute_particle_height(time,
                                initial_velocity,
                                initial_height)
    time_th.append(glob_time)
    velocity_th.append(vel)
    position_th.append(pos)
    
    if (pos<spatial_residual and time>1e-3):   
        time=0
        initial_height=spatial_residual
        initial_velocity=e_dry*abs(vel)
        
    time+=dt
    ind_time+=1
    glob_time+=dt
### data extraction ###
df=pd.DataFrame()
for model, vmodel in collision_model.items(): 
    time,py,uy=np.loadtxt(f"{run.BUILD_DIRECTORY}/dry/{model}/gondret_particles_trajectory_interf.out",skiprows=10,usecols=(0,2,5),unpack=True)
    df[f"time_{model}"]=time
    df[f"py_{model}"]=py-radius
    df[f"uy_{model}"]=uy

### plots ###
# position
fig,ax=plt.subplots()
ax.set_title("Position of the bottom of the particle")
ax.set_ylabel("height [m]", fontsize=15)
ax.set_xlabel(r"$time [s]$", fontsize=15)
for model in collision_model.keys(): 
    ax.plot(df[f"time_{model}"],df[f"py_{model}"],label=f"{model}",lw=1,color=dic_color[model])
ax.plot(time_th,position_th,linewidth=1,color="black", label="analytical")
ax.legend(loc="best")

# velocity
fig,ax=plt.subplots()
ax.set_title("Velocity of the particle")
ax.set_ylabel("Velocity [m/s]", fontsize=15)
ax.set_xlabel(r"$time [s]$", fontsize=15)
for model in collision_model.keys(): 
    ax.plot(df[f"time_{model}"],df[f"uy_{model}"],label=f"{model}",lw=1,color=dic_color[model])
ax.plot(time_th,velocity_th,linewidth=1,color="black", label="analytical")
ax.legend(loc="best")

## Wet collision

The case of the collision in a wet medium is investigated through the experimental study of Gondret et al.
With the current version (TrioCFD-1.9.6), the collision model developed by Breugem is better suited to reproduce the experimental results.
This conclusion may change with the computation of the diffusive terms with an harmonic average (expected in TrioCFD-1.9.7).

In [ ]:
import numpy as np

### data extraction ###
df=pd.DataFrame()
for model, vmodel in collision_model.items(): 
    time,py,uy,fy=np.loadtxt(f"{run.BUILD_DIRECTORY}/wet/{model}/gondret_particles_trajectory_interf.out",skiprows=10,usecols=(0,2,5,8),unpack=True)
    df[f"time_{model}"]=time
    df[f"py_{model}"]=py-radius
    df[f"uy_{model}"]=uy
    df[f"fy_{model}"]=fy

time_gondret_py,py_gondret=np.loadtxt(f"{run.ORIGIN_DIRECTORY}/src/data_exp_gondret_position.txt",unpack=True)
time_gondret_uy,uy_gondret=np.loadtxt(f"{run.ORIGIN_DIRECTORY}/src/data_exp_gondret_velocity.txt",unpack=True)

### plots ###
# position
fig,ax=plt.subplots()
ax.set_title("Position of the bottom of the particle")
ax.set_ylabel("height [m]", fontsize=15)
ax.set_xlabel(r"$time [s]$", fontsize=15)
for model in collision_model.keys(): 
    ax.plot(df[f"time_{model}"],df[f"py_{model}"],label=f"{model}",lw=1,color=dic_color[model])
ax.scatter(time_gondret_py,py_gondret-radius,lw=1,color="black",label="exp gondret")
ax.legend(loc="best")

# velocity
fig,ax=plt.subplots()
ax.set_title("Velocity of the particle")
ax.set_ylabel("Velocity [m/s]", fontsize=15)
ax.set_xlabel(r"$time [s]$", fontsize=15)
for model in collision_model.keys(): 
    ax.plot(df[f"time_{model}"],df[f"uy_{model}"],label=f"{model}",lw=1,color=dic_color[model])
ax.scatter(time_gondret_uy,uy_gondret,lw=1,color="black",label="exp gondret")
ax.legend(loc="best")

# force
fig,ax=plt.subplots()
ax.set_title("Collision force ")
ax.set_ylabel("Fy [N]", fontsize=15)
ax.set_xlabel(r"$time [s]$", fontsize=15)
ax.set_xlim(0.048,0.05)
for model in collision_model.keys(): 
    ax.plot(df[f"time_{model}"],df[f"fy_{model}"],label=f"{model}",lw=1,color=dic_color[model])
ax.legend(loc="best")